In [ ]:
# coding: utf8

#Import

In [ ]:
import os
import sys
import json
import pprint
from pyes import *
from datetime import date, datetime

#Confinguration

#Class project

In [ ]:
class demo(object):
    """
    """
    
    def __init__(self, title='title of the demo', desc='...', Type='bonnus', date=date.today(), html=u'index.html'):
        self.title = title
        self.date = date
        self.source = html
        self.demoType = Type
        self.description = desc
        
        self.tools = self.credits = self.seeAlso =[]
        self.score = 1.0
    
    @classmethod
    def addAttributes(attr, values):
        for v in values:
            if v not in attr: attr.append(v)
                
    def addTools(self, values):
        addAttributes(self.tools, values)
        
    def addCredits(self, values):
        addAttributes(self.credits, values)
    
    def addSeeAlso(self, values):
        addAttributes(self.seeAlso, values)
                
    @property
    def show(self):
        return {u'title': self.title,
           u'description': self.description,
           u'type': self.demoType,
           u'date': self.date.strftime(u'%Y-%m-%d'),
           u'source': self.source,
           u'tools': self.tools,
           u'credits': self.credits,
           u'seeAlso': self.seeAlso
          }


#Loader

### connection

In [ ]:
esConn = ES('127.0.0.1:9200')

In [ ]:
#add mapping
mapping = {}

In [ ]:
d = demo(title='title of the demo N°1', Type='db')

In [ ]:
d = demo(title=u'2012/2013 Paris cars crash', Type=u'graph', html=u'cars_one.html')
d.description = u'Which types of vehicule are involve in accidents for this period?'
d.date = date(2015, 4, 20)

In [ ]:
esConn.index(d.show, 'lmd', 'demo')